In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


Loading the data:

Metdata Lifeline cluster:
/groups/umcg-lifelines/prm02/projects/ov22_0666/dataset_order_202403/
-> Need to be copy to our own local directory for processing!
We copied all the files into our group directory:
/groups/umcg-lifelines/tmp01/projects/ov22_0666/metadata_copy/dataset_order_202403/dataset_order_202403

Blood Metabolome omic:
/groups/umcg-lifelines/rsc01/releases/deep_nmr_data/v1/data/LLD_bloodlipids_nmr.txt.gz

Data directory: 
/groups/umcg-lifelines/prm03/releases

We proccessed DEEP data ourown (Tal), files are in the cluster under:
/groups/umcg-lifelines/tmp01/projects/ov22_0666/personal_directories/talbamberger/data/feature_tables

Linkage file to connect metadata sample id with microbiome sample ids:

/groups/umcg-lifelines/tmp01/projects/ov22_0666/personal_directories/talbamberger/metadata/OV22_00666_deep_linkage_file-v2.csv”

In [2]:
# Mapping different pids:
linkage_mgs = pd.read_csv("/groups/umcg-lifelines/tmp01/projects/ov22_0666/personal_directories/talbamberger/data/linkage_file_MGS.txt", header=None, sep=" ")
linkage_mgs.columns = ['LLDEEP_ID', 'sample_alias']
linkage_mgs['sample_alias'] = linkage_mgs['sample_alias'].str.replace('LL', '', regex=False)

linkage_ega = pd.read_csv("/groups/umcg-lifelines/tmp01/projects/ov22_0666/personal_directories/talbamberger/data/feature_tables/EGA-metadata/sample_file.tsv", sep="\t")
linkage_ega['sample_alias'] = linkage_ega['sample_alias'].str.replace('LL', '', regex=False)

pids_mapping = pd.merge(linkage_mgs, linkage_ega, on="sample_alias", how="inner")

Load microbiome data:

In [3]:
taxonomy_deep_path = '/groups/umcg-lifelines/tmp01/projects/ov22_0666/personal_directories/talbamberger/data/feature_tables/kraken_species_level_taxonomy.tsv'
taxonomy_deep = pd.read_csv(taxonomy_deep_path, sep='\t')

taxonomy_deep = taxonomy_deep.drop(columns=['9002000001328080LL'])

taxonomy_deep = taxonomy_deep.set_index('Taxon').T

taxonomy_deep = taxonomy_deep.reset_index(names='sample')

taxonomy_deep = pd.merge(pids_mapping[["LLDEEP_ID", "file_accession_id"]], taxonomy_deep, 
                         left_on = "file_accession_id", right_on="sample", how="inner").drop(
    columns=['file_accession_id', 'sample']).set_index('LLDEEP_ID')


In [4]:
taxonomy_deep.head()

,JADJOV01 sp016713585,Odoribacter sp905193145,Deinococcus sp000745175,Evtepia sp004551945,Corynebacterium durum,An92 sp009935845,Stercoripulliclostridium_A merdigallinarum,F23-B02 sp900556535,UBA7185 sp900556545,Oxalobacter sp905202055,...,Vibrio sp001874155,Streptosporangium roseum,Bog-1183 sp003136635,Novosphingobium lentum,SOJA01 sp014859805,Bacillus_A cereus,Mycobacterium sp903884065,BMS3BBIN02 sp013151445,Prochlorococcus_A sp003212755,GRL18 sp016097415
LLDEEP_ID,,,,,,,,,,,,,,,,,,,,,
LLDeep_0001,0,71,0,37,108,56,14,638,163,0,...,0,0,0,0,0,0,0,0,0,0
LLDeep_0002,29,0,0,0,1203,0,10,0,59,13,...,0,0,0,0,0,0,0,0,0,0
LLDeep_0006,0,189,0,725,291,79,34,49,30,31,...,0,0,0,0,0,0,0,0,0,0
LLDeep_0007,0,520,0,22,214,18,18,63,37,0,...,0,0,0,0,0,0,0,0,0,0
LLDeep_0010,0,318,0,33233,604,75,15,102,67,15,...,0,0,0,0,0,0,0,0,0,0


In [5]:
taxonomy_deep.shape

(1097, 52648)

In [ ]:
# TODO: Rarefied, calculate RA and filter.

Reading metadata - Blood tests:

In [48]:
metadata_variabels_overview_path = '/groups/umcg-lifelines/tmp01/projects/ov22_0666/metadata_copy/dataset_order_202403/dataset_order_202403/variable_overview.csv'

metadata_variabels_overview = pd.read_csv(metadata_variabels_overview_path)

In [9]:
deep_link = pd.read_csv("/groups/umcg-lifelines/tmp01/projects/ov22_0666/personal_directories/talbamberger/metadata/OV22_00666_deep_linkage_file-v2.csv")
a1v2 = pd.read_csv("/groups/umcg-lifelines/tmp01/projects/ov22_0666/personal_directories/talbamberger/metadata/1a_v_2_results_filtered_to_deep.csv")
vo = pd.read_csv("/groups/umcg-lifelines/tmp01/projects/ov22_0666/personal_directories/talbamberger/metadata/variable_overview.csv")
blood_vo = vo[(vo["subsection"] == "Biospecimen: Blood") & (vo["1a_v_2"] == "x")]["variable_name"].values
a1v2 = pd.merge(a1v2, deep_link,  on = "project_pseudo_id",how='inner')
blood = a1v2.set_index('LLDEEP_ID')[blood_vo]

We want to map blood values to floats, currently some are String. We have $5 value for some.

In [44]:
for col in blood.columns:
    try:
        blood[col].astype(float)
    except:
        print(col)

basoconc_result_all_m_1
basocount_result_all_m_1
basoperc_result_all_m_1
cholesterol_result_all_m_1
creatinine_result_all_m_1
eosinoconc_result_all_m_1
eosinocount_result_all_m_1
eosinoperc_result_all_m_1
erythrocyte_result_all_m_1
glucose_result_all_m_1
hba1cconc_result_all_m_1
hba1cperc_result_all_m_1
hdlchol_result_all_m_1
hematocrit_result_all_m_1
hemoglobin_result_all_m_1
ldlchol_result_all_m_1
leukocyte_result_all_m_1
lymphoconc_result_all_m_1
lymphoperc_result_all_m_1
monoconc_result_all_m_1
monoperc_result_all_m_1
neutroconc_result_all_m_1
neutroperc_result_all_m_1
potassium_result_all_m_1
thrombocyte_result_all_m_1
triglyceride_result_all_m_1


In [25]:
blood.head()

,alat_result_all_m_1,albumin_result_all_m_1,alkphos_result_all_m_1,anticcp_result_all_m_1,antidsdna_result_all_m_1,apolipoa1_result_all_m_1,apolipoa1_result_all_m_2,apolipob100_result_all_m_1,apolipob100_result_all_m_2,asat_result_all_m_1,...,potassium_result_all_m_1,sodium_result_all_m_1,ssapreg_result_all_m_1,t3free_result_all_m_1,t4free_result_all_m_1,thrombocyte_result_all_m_1,triglyceride_result_all_m_1,tsh_result_all_m_1,urea_result_all_m_1,uricacid_result_all_m_1
LLDEEP_ID,,,,,,,,,,,,,,,,,,,,,
LLDeep_0001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,253.000,NaN,NaN,NaN,NaN
LLDeep_0134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,210.000,NaN,NaN,NaN,NaN
LLDeep_1399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,241.000,NaN,NaN,NaN,NaN
LLDeep_0136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,186.000,NaN,NaN,NaN,NaN
LLDeep_0139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,302.000,NaN,NaN,NaN,NaN


In [ ]:
for col in blood.columns:
    print(f"{col}:")
    print(blood[col].dropna().unique())
    print()

In [ ]:
blood.groupby(blood.index).apply(lambda x: x.median(skipna=False))

# blood.groupby(blood.index).apply(lambda x: np.nan if x.isna().all() else .

In [ ]:
# TODO: Re-arrange the blood table. I probably have duplication ids, with nans here and there, we need 
# to merge the duplication ids (so we will get the values)

In [40]:
columns_def = metadata_variabels_overview[metadata_variabels_overview.variable_name.isin(blood.columns)][['variable_name','definition_en']]

In [42]:
columns_def.head()

,variable_name,definition_en
2082,alat_result_all_m_1,alat in lithium heparin tube (u/l)
2083,albumin_result_all_m_1,albumin in lithium heparin tube (g/l)
2084,alkphos_result_all_m_1,alkaline phosphatase in lithium heparin tube (...
2085,anticcp_result_all_m_1,anti-ccp (u/ml)
2086,antidsdna_result_all_m_1,anti-dsdna (u/l)


In [49]:
metadata_variabels_overview.head()

,section,subsection,variable_name,subvariable_name,definition_nl,definition_en,1a_q_1,1a_q_2,1a_q_child,1a_q_youth,...,3a_q_2,3a_q_3,3a_v_1,3a_v_2,3b_q_1,sec_c_dhd,sec_c_ffq,sec_c_foodallergy,sec_c_upfnova,sec_p_participation
0,Demographics,Sex & gender,gender_description_adu_q_1,NaN,zou u hieronder kunnen aangeven welke beschrij...,could you indicate which description fits you ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Diseases & symptoms,Allergies,allergen_animals_adu_q_1,NaN,"dieren (kat, hond, etc.) / kunt u aangeven waa...","animals (cat, dog etc.) / could you indicate w...",x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Diseases & symptoms,Allergies,allergen_contact_adu_q_1,NaN,"contact allergie (nikkel, latex e.d.) / kunt u...","contact allergy (nickel, latex etc.) / could y...",x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Diseases & symptoms,Allergies,allergen_foods_adu_q_1,NaN,"voedingsmiddelen (ei, pinda, etc.) / kunt u aa...","foods (egg, peanut etc.) / could you indicate ...",x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Diseases & symptoms,Allergies,allergen_housedust_adu_q_1,NaN,stof (huisstof e.d.) / kunt u aangeven waar u ...,dust (house dust and the like) / could you ind...,x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
variabels_path = '/groups/umcg-lifelines/tmp01/projects/ov22_0666/metadata_copy/dataset_order_202403/dataset_order_202403/variables/1a_v_2_variables.csv'

variabels = pd.read_csv(variabels_path)

In [59]:
enumberations_path = '/groups/umcg-lifelines/tmp01/projects/ov22_0666/metadata_copy/dataset_order_202403/dataset_order_202403/enumerations/1a_v_2_enumerations.csv'
enumberations = pd.read_csv(enumberations_path)

In [64]:
variabels[variabels['variable_name'].isin(blood.columns)].head()

,variable_name,historic_variable_name,datatype,definition_en,definition_nl,section,subsection,alt_section,alt_subsection,unit
6,alat_result_all_m_1,alt,STRING,alat in lithium heparin tube (u/l),alat in lithium heparine buis (u/l),Physical state,Biospecimen: Blood,NaN,NaN,NaN
7,albumin_result_all_m_1,balb,STRING,albumin in lithium heparin tube (g/l),albumine in lithium heparine buis (g/l),Physical state,Biospecimen: Blood,NaN,NaN,NaN
8,alkphos_result_all_m_1,af,STRING,alkaline phosphatase in lithium heparin tube (...,alkaline phosphatase in lithium heparine buis ...,Physical state,Biospecimen: Blood,NaN,NaN,NaN
9,anticcp_result_all_m_1,ccp,STRING,anti-ccp (u/ml),anti-ccp (u/ml),Physical state,Biospecimen: Blood,NaN,NaN,NaN
10,antidsdna_result_all_m_1,ddna,STRING,anti-dsdna (u/l),anti-dsdna (u/l),Physical state,Biospecimen: Blood,NaN,NaN,NaN


In [65]:
results_path = '/groups/umcg-lifelines/tmp01/projects/ov22_0666/metadata_copy/dataset_order_202403/dataset_order_202403/results/1a_v_2_results.csv'
results = pd.read_csv(results_path)

/local/3413276/ipykernel_4770/869431256.py:2: DtypeWarning: Columns (16,17,18,23,24,26,27,28,29,30,32,33,34,35,36,37,38,41,42,43,45,46,47,48,49,50,52,53,57,58,63,64,65,66,67,68,69,70,72,77,78,79,80,81,82,83) have mixed types. Specify dtype option on import or set low_memory=False.
  results = pd.read_csv(results_path)


In [66]:
results.head()

,project_pseudo_id,variant_id,date,age,gender,zip_code,alat_result_all_m_1,albumin_result_all_m_1,alkphos_result_all_m_1,anticcp_result_all_m_1,...,uremsteststrip_nitrite_all_m_1,uremsteststrip_ph_all_m_1,uremsteststrip_protein_all_m_1,urems_ur24h_all_m_1,uricacid_result_all_m_1,urtoalbumin_result_all_m_1,urto_blood_all_m_1,urtocreatinine_result_all_m_1,urto_missed_all_m_1,urto_volume_all_m_1
0,44e1b77e-6a72-4ed0-9561-c19c2cd1997f,1a_v_2_bloodcells1_8plus_v1,2007-02,43,MALE,8604.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34be398c-7ca5-4691-bb95-8feaefdd9781,1a_v_2_bloodcells1_8plus_v1,2011-09,34,FEMALE,8604.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,942b3d98-c3d3-4095-8f3a-8fe3bdcfb211,1a_v_2_bloodcells1_8plus_v1,2007-05,41,FEMALE,8605.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6534890e-bf36-4b11-9f9f-418156f4a487,1a_v_2_bloodcells1_8plus_v1,2013-05,41,MALE,8606.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,96ca0e79-bd5a-48ce-9692-ecf4bb9d5eda,1a_v_2_bloodcells1_8plus_v1,2013-07,37,FEMALE,8602.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
results.columns

Index(['project_pseudo_id', 'variant_id', 'date', 'age', 'gender', 'zip_code',
       'alat_result_all_m_1', 'albumin_result_all_m_1',
       'alkphos_result_all_m_1', 'anticcp_result_all_m_1',
       'antidsdna_result_all_m_1', 'apolipoa1_result_all_m_1',
       'apolipoa1_result_all_m_2', 'apolipob100_result_all_m_1',
       'apolipob100_result_all_m_2', 'asat_result_all_m_1',
       'basoconc_result_all_m_1', 'basocount_result_all_m_1',
       'basoperc_result_all_m_1', 'blood_urine24h_all_m_1',
       'blood_urineems_all_m_1', 'blood_urineto_all_m_1',
       'calcium_result_all_m_1', 'cholesterol_result_all_m_1',
       'creatinine_result_all_m_1', 'ctdscreen_result_all_m_1',
       'eosinoconc_result_all_m_1', 'eosinocount_result_all_m_1',
       'eosinoperc_result_all_m_1', 'erythrocyte_result_all_m_1',
       'fasting_participant_all_q_1', 'gammagt_result_all_m_1',
       'glucose_result_all_m_1', 'hba1cconc_result_all_m_1',
       'hba1cperc_result_all_m_1', 'hdlchol_result_all

In [60]:
enumberations

,variable_name,enumeration_code,enumeration_en,enumeration_nl
0,blood_urine24h_all_m_1,5,24h urine not obtained,24-uurs urine niet afgenomen
1,blood_urine24h_all_m_1,1,yes,ja
2,blood_urine24h_all_m_1,3,no,nee
3,blood_urine24h_all_m_1,2,"likely yes, but one or both of the dates were ...","waarschijnlijk ja, maar een of beide datums zi..."
4,blood_urine24h_all_m_1,4,"likely no, but one or both of the dates were n...","waarschijnlijk nee, maar een of beide datums z..."
...,...,...,...,...
93,urto_blood_all_m_1,4,"likely no, but one or both of the dates were n...","waarschijnlijk nee, maar een of beide datums z..."
94,urto_missed_all_m_1,3,"yes, 1 urination was missed","ja, er is 1 plas gemist"
95,urto_missed_all_m_1,4,"yes, 2 urinations or more were missed","ja, er zijn 2 of meer plassen gemist"
96,urto_missed_all_m_1,2,"no, all urinations were collected","nee, alle urine is opgevangen"


In [ ]:
var_name = 'basoconc_result_all_m_1'